<a href="https://colab.research.google.com/github/NikolayWTF/Hackathon-Perm/blob/main/%D0%A5%D0%B0%D0%BA%D0%B0%D1%82%D0%BE%D0%BD_%D0%9F%D0%B5%D1%80%D0%BC%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Детекция и подсчёт ТБО
## Выбор нейросети
Модель yolov8m показала наилучший результат при тесте. М1 = 0.95.

При желании вы можете протестировать другие модели. Достаточно просто раскоментировать строку с интересующей вас моделью.

In [ ]:
# !gdown 1BlUQpPX0VeOBHWtXH0uRB3HVI_BD6CsT # YOLOv8m + аугментация M1 = 1.25
# !gdown 1X9ZySLYXzoidGfGc5XThHu8X1xaz4t7P # yolov8n_150epochs.pt M1 = 1.16
# !gdown 1AIIIKnNWGWTxIRdEw1OzWNeIs7h6QOdV # yolov8s_180epochs.pt M1 = 1.13
# !gdown 1QEtK_0D09Xo1Le2s16DjOM6oP8KfasgS # yolov8_large_best.pt M1 = 1
# !gdown 1oq_ZNquGsW2nvIxxewizq9kMLJjUhDXt # yolov8m - для тех скора.
!gdown 1JSFVsos9kUgOYesKEK-YM-qnzUIWSLig # yolov8m_100epochs.pt M1 = 0.95
!gdown 1SLSp9FD8VBOkLY0gGoAk0TFsLc7StvYD # видео

In [ ]:
!pip install unrar
!pip install ultralytics # Устанавливаю YOLO

In [ ]:
# Распаковываю архив с фреймами и добавляю в папку video
!mkdir "video"
!unrar x -Y "video0.rar" "video"

In [6]:
import ultralytics
import os
import math

# Составляем ответ для технического скора

In [13]:
!mkdir frames_output # Создаём папку для хранения ответов

In [ ]:
#Генерация ответа
# Выбираем модель
model = ultralytics.YOLO("yolov8m_100epochs.pt")
path = "video/frames_rgb_0/"
valid = ["wood", "glass", "plastic", "metal"]

for file in os.listdir(path): # Проходим по всем фреймам
    file_path = path + file
    result = model(file_path)  # Детекция ТБО для фрейма

    local_TBO = {
        "wood": 0,
        "glass": 0,
        "plastic": 0,
        "metal": 0,
    }

    for r in result: # Перебираем по 1 распознаному объекту
        boxes = r.boxes
        for box in boxes:
            name = model.names[int(box.cls)]  # Получаем название класса ТБО
            if name in valid: # Если распознали ТБО
                local_TBO[name] += 1 # Обновляем их количество для соответствующего класса
    # Записываем ответ в файл
    ans = ""
    for key in valid:
        ans += str(local_TBO[key]) + "\n"
    f = open("frames_output/" + file[:-3] + "txt", mode='w')
    f.write(ans[:-1])
    f.close()


# Вычисление метрики М1

In [ ]:
# Вычисление метрики M1

# Функция для расчёта подкоренной суммы RMSD
def count_RMSD_sum(key):
  return (ans_TBO[key] - local_TBO[key])**2

# Укажите путь к модели
model = ultralytics.YOLO("yolov8m_100epochs.pt")
# Укажите путь к фреймам
path = "video/frames_rgb_0/"

valid = ["wood", "glass", "plastic", "metal"]
RMSD_w, RMSD_p, RMSD_g, RMSD_m = 0, 0, 0, 0
T = len(os.listdir(path))

for file in os.listdir(path): # Проходим по всем фреймам
    file_path = path + file
    result = model(file_path) # Детекция ТБО для фрейма

    # Из файла с ответами узнаём сколько на фрейме ТБО и какие они
    file_ans = open("video/frames_output/" + file[:-3] + "txt", "r")
    lines = file_ans.readlines()
    ans_TBO = {
        "wood": int(lines[0]),
        "glass": int(lines[1]),
        "plastic": int(lines[2]),
        "metal": int(lines[3]),
    }
    file_ans.close
    # Тут хранится ТБО для данного фрейма
    local_TBO = {
        "wood": 0,
        "glass": 0,
        "plastic": 0,
        "metal": 0,
    }

    for r in result: # Перебираем по 1 распознаному ТБО
        boxes = r.boxes
        for box in boxes:
            name = model.names[int(box.cls)] # Получаем название класса ТБО
            if name in valid: # Проверяем, что YOLO нашла ТБО, а не что-то другое
                local_TBO[name] += 1
    # Считаем подкоренную сумму для каждого из классов
    RMSD_w += count_RMSD_sum("wood")
    RMSD_g += count_RMSD_sum("glass")
    RMSD_p += count_RMSD_sum("plastic")
    RMSD_m += count_RMSD_sum("metal")
# Делим на количество фреймов и извлекаем корень
RMSD_w = math.sqrt(RMSD_w/T)
RMSD_m = math.sqrt(RMSD_m/T)
RMSD_g = math.sqrt(RMSD_g/T)
RMSD_p = math.sqrt(RMSD_p/T)
# Получаем среднее значение - это и есть M1
M1 =(RMSD_w + RMSD_p + RMSD_g + RMSD_m)/4

In [8]:
M1

0.949910625570183

# Вычисление дублирующего коэфициента для метрики М2
Подробнее про дублирующий коэфициент можно прочитать в репозитории

In [ ]:
# Вычисление дублирующего коэффициента

# Укажите путь к модели
model = ultralytics.YOLO("yolov8m_100epochs.pt")
path = "video/frames_rgb_0/"

valid = ["wood", "glass", "plastic", "metal"]

# Тут хранится итоговое количество ТБО
global_TBO = {
        "wood": 0,
        "glass": 0,
        "plastic": 0,
        "metal": 0,
          }
# Тут хранится количество ТБО, которые были на линии на предидущем фрейме
prev_TBO = {
        "wood": 0,
        "glass": 0,
        "plastic": 0,
        "metal": 0,
          }

# Вертикальная черта на соответствующих координатах.
lines = [100, 200, 300, 400, 500, 600] # Уровень для линии - 100, 200, 300, 400, 500, 600
T = len(os.listdir(path)) # Кол-во фреймов
for line in lines:
    for file_ind, file in enumerate(os.listdir(path)): # Проходим по всем фреймам
        file_path = path + file
        result = model(file_path) # Детекция ТБО для фрейма

        for r in result: # Перебираем по 1 распознаному ТБО
            local_TBO = {
            "wood": 0,
            "glass": 0,
            "plastic": 0,
            "metal": 0,
            }
            boxes = r.boxes
            for box in boxes:
                x_0 = int(box.xyxy[0][0])
                x_1 = int(box.xyxy[0][2])
                name = model.names[int(box.cls)] # Получаем название класса ТБО
                if name in valid: # Если распознали ТБО
                    if file_ind == 0: # Если это первый фрейм
                        if x_0 > line: # То всё что после линии добавляем к ответу
                            global_TBO[name] += 1
                        elif (x_0 < line) and (x_1 > line): # А всё что на линии записываем в prev_TBO
                            prev_TBO[name] += 1
                    elif file_ind == T-1: # Если это последний фрейм, то всё что перед или на линии добавляем к ответу
                        if (x_0 <= line):
                            global_TBO[name] += 1
                    else:  # Если это все остальные фреймы
                      if (x_0 < line) and (x_1 > line): # Считаем количество ТБО на линии
                        local_TBO[name] += 1
                        for key in valid:
                          # Сравниваем с предидущим фреймом и обновляем число уникальных ТБО
                          global_TBO[key] += max(prev_TBO[key] - local_TBO[key], 0)
                          prev_TBO[key] = local_TBO[key] # Обнавляем количество ТБО на прошлом фрейме


In [10]:
ans = {
    "wood" : 67,
    "glass" : 59,
    "plastic" : 57,
    "metal" : 61
}

koeff = {}

for key in valid:
    global_TBO[key]/=6 # Так как у нас 6 линий, то нужно взять среднее.
    koeff[key] = ans[key] / global_TBO[key] # Дублирующий коэфициент
print(global_TBO)


{'wood': 131.16666666666666, 'glass': 126.0, 'plastic': 179.0, 'metal': 67.16666666666667}


In [11]:
koeff # Дублирующие коэффициенты

{'wood': 0.5108005082592122,
 'glass': 0.46825396825396826,
 'plastic': 0.31843575418994413,
 'metal': 0.9081885856079404}

In [16]:
f = open("output.txt", 'w')
ans = ""
for key in valid:
    ans += str(round(global_TBO[key] * koeff[key])) + "\n"
f.write(ans[:-1])
f.close()

# Визуализация полученых данных.
### Этот блок можно использовать после получения папки с ответами.
Ответы можно получить, выполнив код в ячейке "Составляем ответ для технического скора"

In [ ]:
!pip install ipywidgets

In [27]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import output
output.enable_custom_widget_manager()

In [116]:
import plotly.graph_objs as go
import ipywidgets as widgets

def rebinnable_interactive_histogram():
    figure_widget = go.FigureWidget(
        dict({
          "data": [{"type": "bar",
                    "x": ["wood", "glass", "plastic", "metal"],
                    "y": [3, 4, 7, 2]}],
          "layout": {"title": {"text": "Распределение ТБО на 0000 фрейме"},
          "template": "ggplot2"}
        })
    )

    slider = widgets.IntSlider(
    value=0,
    min=0,
    max=334,
    step=1,
    readout_format='.0f',
    )

    histogram_object = figure_widget.data[0]
    histogram_object2 = figure_widget.layout["title"]

    def set_bin_size(change):
        num = str(change["new"])
        while len(num) < 4:
            num = "0" + num

        filename = "frames_output/" + num + ".txt"
        f = open(filename, "r")
        lines = f.readlines()
        y = [int(lines[0]), int(lines[1]), int(lines[2]), int(lines[3])]
        file_ans.close
        histogram_object["y"] = y
        histogram_object2["text"] = "Распределение ТБО на "+ num +  " фрейме"

    slider.observe(set_bin_size, names="value")

    output_widget = widgets.VBox([figure_widget, slider])

    return output_widget

rebinnable_interactive_histogram()


    'data': [{'type': 'bar',
              'uid': '55f45cef-ed11-42eb-86df-6abe1…

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()